In [1]:
from broyden import *
from solver_helper import *

import matplotlib.pyplot as plt 
import pandas as pd
import time

In [2]:
#rmf = eos(g_sigma_N = 8.79509376389, g_omega_N = 9.1815177, g_rho_N = 9.7793745, b = 0.00414, c = 0.00716)
rmf = eos(g_sigma_N = 8.784820, g_omega_N = 8.720086, g_rho_N = 8.544795, b = 0.008628, c = -0.002433)

baryon_list = [Neutron, Proton]
meson_list = [sigma, omega, rho]
lepton_list = [electron]

In [3]:
B = full_solve(rmf, baryon_list, lepton_list, meson_list, [8.0, 4.50, -2.2, 210.0, 43.0, 43.0])
B

nB/n0  sigma field (MeV)  rho field (MeV)  omega field (MeV)  \
0    0.27           7.941193        -2.247962           4.514730   
1    0.28           8.199099        -2.329050           4.681942   
2    0.29           8.455030        -2.409949           4.849154   
3    0.30           8.709030        -2.490655           5.016367   
4    0.31           8.961139        -2.571167           5.183579   
..    ...                ...              ...                ...   
88   1.15          25.791266        -8.542271          19.229406   
89   1.16          25.956326        -8.603948          19.396618   
90   1.17          26.120786        -8.665421          19.563830   
91   1.18          26.284650        -8.726690          19.731042   
92   1.19          26.447924        -8.787756          19.898255   

    Neutron kF (MeV)  Proton kF (MeV)  electron kF (MeV)  Neutron frac  \
0         210.426938        43.238314          43.238314      0.991399   
1         212.960614        44.528049          44.528049      0.990942   
2         215.432600        45.809033          45.809033      0.990477   
3         217.846350        47.081534          47.081534      0.990006   
4         220.205018        48.345801          48.345801      0.989528   
..               ...              ...                ...           ...   
88        334.909950       135.090123         135.090123      0.938414   
89        335.801379       135.949166         135.949166      0.937773   
90        336.686864       136.804973         136.804973      0.937132   
91        337.566492       137.657567         137.657567      0.936492   
92        338.440346       138.506970         138.506970      0.935853   

    Proton frac  electron frac  
0      0.008601       0.008601  
1      0.009058       0.009058  
2      0.009523       0.009523  
3      0.009994       0.009994  
4      0.010472       0.010472  
..          ...            ...  
88     0.061586       0.061586  
89     0.062227       0.062227  
90     0.062868       0.062868  
91     0.063508       0.063508  
92     0.064147       0.064147  

[93 rows x 10 columns]

In [12]:
nb_array = B.loc[:,'nB/n0'].values

sigma_array = B.loc[:,'sigma field (MeV)'].values
rho_array = B.loc[:,'rho field (MeV)'].values
omega_array = B.loc[:,'omega field (MeV)'].values

neutron_frac = B.loc[:, 'Neutron frac'].values
proton_frac = B.loc[:, 'Proton frac'].values
electron_frac = B.loc[:, 'electron frac'].values

fermi_n = B.loc[:, 'Neutron kF (MeV)'].values
fermi_p = B.loc[:, 'Proton kF (MeV)'].values
fermi_e = B.loc[:, 'electron kF (MeV)'].values

## Chemical Potential

In [4]:
def partial_U(sigma_field):
    term1 = rmf.b * Neutron.num_mass * rmf.g_sigma_N**3 * sigma_field**2
    term2 = rmf.c * rmf.g_sigma_N**4 * sigma_field**3 
    return term1 + term2

In [5]:
def eff_mass(baryon, sigma_field):
    return baryon.num_mass - rmf.g_sigma_N * sigma_field

In [6]:
def ef(baryon, kf, sigma_field):
    m = eff_mass(baryon, sigma_field)
    return np.sqrt(kf**2 + m**2) 

In [7]:
def log_fac(baryon, kf, sigma_field):
    m = eff_mass(baryon, sigma_field) 
    EF = ef(baryon, kf, sigma_field)
    
    numerator = kf + EF 
    return np.log(numerator/m) 

In [42]:
def scalar_densityy(baryon, kf, sigma_field):
    m = eff_mass(baryon, sigma_field) 
    EF = ef(baryon, kf, sigma_field) 
    log_factor = log_fac(baryon, kf, sigma_field)
    
    term1 = EF * kf 
    term2 = - m**2 * log_factor
    
    return m/(2*np.pi**2)*(term1 + term2)

In [43]:
def scalar_density(baryon):
    # returns scalar density n_s for a given baryon
    # note: we modify the argument of the natural log as to be positive definite to avoid complex numbers 
    # this is a symbolic expression 

    coeff_1 = (2*baryon.spin + 1)/(2*Pi**2)
    coeff_2 = baryon.sym_g_sigma*baryon.sym_mass_eff
    term_2 = baryon.sym_ef*baryon.sym_kf 
    term_2_2 = sym.sqrt(((baryon.sym_ef + baryon.sym_kf)/baryon.sym_mass_eff)**2)
    term_3 = sym.log(term_2_2)
    
    return coeff_1*coeff_2*(term_2 - baryon.sym_mass_eff**2*term_3)

## Chemical Potential

In [4]:
def alpha_i(baryon, kf, sigma_field):
    m = eff_mass(baryon, sigma_field) 
    EF = ef(baryon, kf, sigma_field) 
    log_factor = log_fac(baryon, kf, sigma_field) 
    g_sigma = baryon.num_g_sigma 
    
    term1 = - g_sigma/(2* np.pi**2) * (kf * EF - m**2 * log_factor) 
    
    term2_a = - baryon.num_g_sigma * kf * m / EF 
    term2_b = 2 * m * g_sigma * log_factor 
    term2_c = - g_sigma * m**3 / EF / (kf + EF) 
    term2_d = m * g_sigma 
    
    term2 = m/(2*np.pi**2)*(term2_a + term2_b + term2_c + term2_d) 
    
    return term1 + term2

In [5]:
def beta_i(baryon, kf, sigma_field):
    m = eff_mass(baryon, sigma_field) 
    EF = ef(baryon, kf, sigma_field) 
    g_sigma = baryon.num_g_sigma 
    
    term1 = EF
    term2 = kf**2 / EF 
    term3 = m**2 /EF
    
    return m/(2*np.pi**2)*(term1 + term2 + term3) 

In [6]:
def sec_deriv(sigma_field):
    term1 = 2 * sigma.num_b * Neutron.num_mass * Neutron.num_g_sigma **3 * sigma_field 
    term2 = 3 * sigma.num_c * Neutron.num_g_sigma**4 * sigma_field**2 
    return term1 + term2 

In [7]:
def partial_sigma_partial_kf(Neutron, Proton, n_kf, p_kf, sigma_field):
    m = eff_mass(Neutron, sigma_field) 
    n_EF = ef(Neutron, kf, sigma_field)
    p_EF = ef(Proton, kf, sigma_field) 
    
    g_sigma = Neutron.num_g_sigma 
    
    numerator = g_sigma * (beta_i(Neutron, n_kf, sigma_field) + beta_i(Proton, p_kf, sigma_field)) 
    
    denominator = sigma.num_mass**2 + sec_deriv(sigma_field) 
    
    denominator += g_sigma * alpha_i(Proton, p_kf, sigma_field) 
    denominator += g_sigma * alpha_i(Neutron, n_kf, sigma_field)
    
    return numerator/denominator 

In [ ]:
partial_sigma_kfp = np.zeros(len(nb_array)) 
partial_sigma_kfn = np.zeros(len(nb_array)) 

for i in range(len(partial_sigma_kfn)):
    partial_sigma_kfp[i] = partial_sigma_partial_kf(Neutron, Proton, fermi_n[i], fermi_p[i], sigma_array[i])

In [8]:
sigma.num_b

0.008628

## Sigma Field Partial Derivative

In [10]:
def d_U(sigma_field):
    term1 = sigma.num_b * Neutron.num_mass * Neutron.num_g_sigma**3 * sigma_field**2 
    term2 = sigma.num_c * Neutron.num_g_sigma**4 * sigma_field**3 
    return term1 + term2 

In [9]:
def scalar_dens(fermi, sigma_field, baryon):
    m = baryon.num_mass - baryon.num_g_sigma * sigma_field 
    EF = np.sqrt(fermi**2 + m**2) 
    log_fac = np.log((EF + fermi)/m)
    
    term1 = EF * kf 
    term2 = - m**2 * log_fac
    return m/(2*np.pi**2)*(term1 + term2) 

In [11]:
def sigma_eom(n_fermi, p_fermi, sigma_field):
    return sigma.num_mass**2 * sigma_field + d_U(sigma_field)\
            - Neutron.num_g_sigma * scalar_dens(n_fermi, sigma_field, Neutron)\
            - Proton.num_g_sigma * scalar_dens(p_fermi, sigma_field, Proton)